In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
import spacy
import csv
from time import time
from IPython.display import clear_output

In [2]:
# define binary search to used in remNonSkills
def bs(target, arr):
    lo = 0
    hi = len(arr)-1
    while lo <= hi:
        mid = (lo + hi)//2
        if arr[mid] == target:
            return True
        elif target < arr[mid]:
            hi = mid-1
        else:
            lo = mid+1
    return False 

# define function to remove stop words from a type doc resume and returns a new doc
def remStopWords(docParam):
    # remove stopwords
    stopwords = nlp.Defaults.stop_words
    text = docParam.text
    lst=[]
    for token in text.split():
        if token.lower() not in stopwords:    #checking whether the word is not 
            lst.append(token)                    #present in the stopword list.
    return nlp(' '.join(lst).lower())    

# define a function that splits a type doc resume with stop words removed into base noun phrases 
def chunkSplit(doc):
    chunks = set()
    individual_words = set()
    for chunk in doc.noun_chunks:
        chunks.add(chunk.text)
        individual_words.add(chunk.root.head.text)
        
    clean_chunks = []  
    for chunk in chunks: 
        clean_chunks.extend(re.split(r"(, | － )", chunk))
    return clean_chunks, individual_words

# define a function that takes a cleaned array and removes any words that are not in the skills set
# returns a dataframe
def remNonSkills(df, docParam, chunksParam, individual_words, i, skills, column): 
    ind = 0
    removeWords = []
    chunksParam.extend(list(individual_words))
    for word in chunksParam:
        if(not bs(word, skills)):
            removeWords.append(word)

    #write new resume to dataframe
    resume = ' '.join([_word for _word in chunksParam if _word not in removeWords])
    df.loc[i, column] = resume
    return df

# Combine all cleaning functions into one function call
def cleanResume(df,i, skills, column):
    doc = nlp(df.loc[i, column])
    doc = remStopWords(doc)
    clean_chunks,individual_words=chunkSplit(doc)
    remNonSkills(df,doc,clean_chunks,individual_words,i,skills,column)

# Clean jobs data set and combine job description and required skills
def refineData(df):
    #drop 2 columns
    df = df.drop(columns = ['company_review'])
    df = df.drop(columns =['salary_offered'])
    #merge two columns
    df["Description + Skills"] = df['job_description'].astype(str) +": Skill List: "+ df['required_skills']
    return df

In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
df = pd.read_csv("./resume.csv")

In [5]:
# read linkedin keywords
filename = "./linkedinskill"
f = open(filename, 'rb')
Lines = f.readlines()
skills = []
for line in Lines:
    line = line[:-1].decode("utf-8")
    skills.append(line[:-1].lower())
# skills

In [6]:
# Clean resumes based on Linkedin keywords
t0 = time()
for i in range(len(df)):
    cleanResume(df,i, skills, 'Resume_str')
    clear_output(wait=True)
    print(f"processing resume index: {i}")
duration = time() - t0
clear_output(wait=True)
print(f"done in {duration:.3f} s")

done in 697.293 s


In [7]:
# TODO: find intersection of job&resume vocab
# fit to intersection
# transform content
# sum across rows (min of (1, x)) to find mean
# describe combined dataframe

# find overlap between resume and job_des. 
# count vectorize that

# find cosine similarity


# Count Vectorize cleaned resumes
t0 = time()
content = df["Resume_str"]

# Create a Vectorizer Object
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(content)
vector = vectorizer.transform(content)
vector_features = vectorizer.get_feature_names_out()
vector_arr = vector.toarray()
count_vect_df = pd.DataFrame(vector.todense(), columns=vector_features)
# df = pd.concat([df, count_vect_df.reset_index(drop=True)])
duration = time() - t0


print("Vocabulary: ", len(vectorizer.vocabulary_))
print(f"done in {duration:.3f} s")
print(f"Found {len(vectorizer.get_feature_names_out())} unique terms")
print("Encoded Document is:")
print(vector)
vectorizer.vocabulary_

Vocabulary:  3708
done in 0.155 s
Found 3708 unique terms
Encoded Document is:
  (0, 135)	1
  (0, 820)	1
  (0, 931)	1
  (0, 1278)	1
  (0, 1344)	1
  (0, 1383)	1
  (0, 1624)	1
  (0, 1724)	1
  (0, 1751)	1
  (0, 1759)	1
  (0, 1998)	1
  (0, 2125)	1
  (0, 2166)	1
  (0, 2187)	2
  (0, 2376)	1
  (0, 2380)	2
  (0, 2407)	1
  (0, 2481)	1
  (0, 2497)	1
  (0, 2584)	1
  (0, 2625)	3
  (0, 2661)	1
  (0, 2715)	3
  (0, 2972)	1
  (0, 3100)	3
  :	:
  (2483, 267)	1
  (2483, 624)	1
  (2483, 792)	2
  (2483, 1026)	1
  (2483, 1095)	1
  (2483, 1138)	1
  (2483, 1180)	1
  (2483, 1819)	1
  (2483, 1998)	1
  (2483, 2044)	1
  (2483, 2097)	1
  (2483, 2181)	1
  (2483, 2228)	1
  (2483, 2356)	1
  (2483, 2376)	1
  (2483, 2495)	1
  (2483, 2497)	2
  (2483, 2670)	1
  (2483, 2805)	1
  (2483, 2890)	1
  (2483, 2922)	1
  (2483, 3070)	1
  (2483, 3269)	1
  (2483, 3630)	1
  (2483, 3648)	3


{'paperwork': 2125,
 'records': 2584,
 'training': 3386,
 'health': 1344,
 'statistics': 3100,
 'time': 3330,
 'management': 1751,
 'productivity': 2407,
 'holidex': 1383,
 'performance': 2187,
 'reviews': 2715,
 'loss': 1724,
 'prevention': 2380,
 'analytical': 135,
 'skills': 2972,
 'documentation': 820,
 'employee': 931,
 'relations': 2625,
 'government': 1278,
 'layout': 1624,
 'office': 1998,
 'payroll': 2166,
 'presentations': 2376,
 'public': 2481,
 'purchasing': 2497,
 'reporting': 2661,
 'mar': 1759,
 'asset': 236,
 'graphics': 1293,
 'lotus': 1726,
 'notes': 1974,
 'microsoft': 1819,
 'publications': 2482,
 'web': 3600,
 'design': 711,
 'articles': 214,
 'union': 3489,
 'events': 1020,
 'launches': 1617,
 'photoshop': 2237,
 'staff': 3076,
 'publisher': 2484,
 'workers': 3652,
 'compensation': 549,
 'pagemaker': 2101,
 'organizational': 2050,
 'regulatory': 2616,
 'compliance': 552,
 'harvard': 1334,
 'excel': 1026,
 'quattro': 2523,
 'pro': 2397,
 'strategic': 3131,
 'planni

In [8]:
#save vocabulary to text file
import json
with open('ResumeCountVecVocab.txt', 'w') as convert_file:
     convert_file.write(json.dumps(vectorizer.vocabulary_))

In [9]:
count_vect_df

,10,1120s,12c,133,1x,2008,2012,21,25,264,...,zebra,zemax,zendesk,zenworks,zeta,zoho,zoning,zoom,zumba,zynx
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
count_vect_df.describe()

,10,1120s,12c,133,1x,2008,2012,21,25,264,...,zebra,zemax,zendesk,zenworks,zeta,zoho,zoning,zoom,zumba,zynx
count,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,...,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000,2484.000000
mean,0.004026,0.000403,0.000403,0.000403,0.001208,0.001208,0.000805,0.000805,0.000403,0.004026,...,0.000805,0.000805,0.002818,0.000805,0.000403,0.002013,0.000805,0.000403,0.002415,0.000403
std,0.089658,0.020064,0.020064,0.020064,0.044858,0.034738,0.028370,0.028370,0.020064,0.085048,...,0.040129,0.040129,0.072303,0.028370,0.020064,0.060171,0.028370,0.020064,0.063416,0.020064
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000


In [11]:
# read job listing dataset
jobs = pd.read_csv("./JobsData.csv")
jobs = refineData(jobs)

In [12]:
# clean job listing dataset based on Linkedin keywords
t0 = time()
for i in range(len(jobs)):
    cleanResume(jobs,i, skills, "Description + Skills")
    clear_output(wait=True)
    print(f"processing job posting index: {i}")
duration = time() - t0
clear_output(wait=True)
print(f"done in {duration:.3f} s")

done in 740.634 s


In [13]:
jobs["Description + Skills"][1000:2000]

1000                 
1001    design design
1002            rdbms
1003          project
1004                 
            ...      
1995                 
1996                 
1997                 
1998                 
1999                 
Name: Description + Skills, Length: 1000, dtype: object

In [14]:
# Count Vectorize cleaned jobs
t0 = time()
content = jobs["Description + Skills"]

# Create a Vectorizer Object
vectorizerJobs = CountVectorizer(min_df=1)
vectorizerJobs.fit(content)
vector = vectorizerJobs.transform(content)
vector_features = vectorizerJobs.get_feature_names_out()
vector_arr = vector.toarray()
count_vect_jobs = pd.DataFrame(vector.todense(), columns=vector_features)
duration = time() - t0

print("Vocabulary: ", len(vectorizerJobs.vocabulary_))
print(f"done in {duration:.3f} s")
print(f"Found {len(vectorizerJobs.get_feature_names_out())} unique terms")
print("Encoded Document is:")
print(vector)
vectorizerJobs.vocabulary_

Vocabulary:  1179
done in 0.323 s
Found 1179 unique terms
Encoded Document is:
  (0, 834)	1
  (8, 1147)	1
  (9, 860)	1
  (11, 1087)	1
  (14, 278)	1
  (14, 582)	2
  (18, 1087)	1
  (22, 636)	1
  (22, 950)	1
  (23, 1088)	1
  (25, 227)	1
  (26, 343)	1
  (26, 1030)	1
  (26, 1065)	1
  (30, 1087)	1
  (30, 1088)	1
  (32, 491)	1
  (33, 490)	1
  (34, 24)	1
  (34, 62)	1
  (34, 248)	1
  (36, 666)	1
  (38, 745)	1
  (38, 787)	1
  (39, 1106)	1
  :	:
  (32687, 315)	1
  (32687, 323)	2
  (32687, 874)	1
  (32691, 889)	1
  (32695, 227)	1
  (32695, 243)	1
  (32695, 248)	1
  (32695, 745)	1
  (32695, 1135)	1
  (32701, 834)	1
  (32704, 42)	1
  (32705, 845)	1
  (32707, 845)	1
  (32710, 322)	1
  (32710, 1158)	1
  (32721, 1155)	2
  (32731, 1155)	2
  (32732, 1026)	1
  (32733, 824)	1
  (32733, 964)	1
  (32734, 430)	1
  (32734, 860)	1
  (32735, 824)	1
  (32735, 860)	1
  (32737, 824)	1


{'resolve': 834,
 'windows': 1147,
 'sap': 860,
 'troubleshooting': 1087,
 'microsoft': 582,
 'dynamics': 278,
 'open': 636,
 'source': 950,
 'tuning': 1088,
 'debugging': 227,
 'fault': 343,
 'tolerant': 1065,
 'systems': 1030,
 'jenkins': 491,
 'java': 490,
 'agile': 24,
 'application': 62,
 'development': 248,
 'pages': 666,
 'product': 745,
 'quality': 787,
 'unix': 1106,
 'linux': 543,
 'hiring': 442,
 'databases': 223,
 'mongodb': 598,
 'cs': 205,
 'research': 831,
 'design': 243,
 'microservices': 581,
 'hana': 430,
 'architecture': 72,
 'technical': 1041,
 'writing': 1163,
 'leadership': 525,
 'python': 779,
 'spark': 954,
 'reporting': 824,
 'fusion': 402,
 'crm': 203,
 'technology': 1047,
 'crts': 204,
 'automation': 95,
 'spring': 966,
 'mvc': 607,
 'boot': 130,
 'looker': 555,
 'telecom': 1050,
 'electronics': 300,
 'vue': 1131,
 'government': 417,
 'proof': 757,
 'mysql': 608,
 'test': 1054,
 'cases': 146,
 'patterns': 674,
 'rdbms': 795,
 'analytical': 45,
 'skills': 930,

In [15]:
#save vocabulary to text file
import json
with open('JobsCountVecVocab.txt', 'w') as convert_file:
     convert_file.write(json.dumps(vectorizerJobs.vocabulary_))

In [17]:
tempdf = pd.read_csv('./IntersectionVocab.txt')
t0 = time()

# Create a Vectorizer Object for Resume
resume_content = df["Resume_str"]
Iresume_vectorizer = CountVectorizer(min_df=1)
Iresume_vectorizer.fit(tempdf)
Iresume_vector = Iresume_vectorizer.transform(resume_content)
Iresume_vector_features = Iresume_vectorizer.get_feature_names_out()
Iresume_count_vect_df = pd.DataFrame(Iresume_vector.todense(), columns=Iresume_vector_features)

# Create a Vectorizer Object for Joblistings
jobs_content = jobs["Description + Skills"]
Ijobs_vectorizer = CountVectorizer(min_df=1)
Ijobs_vectorizer.fit(tempdf)
Ijobs_vector = Ijobs_vectorizer.transform(jobs_content)
Ijobs_vector_features = Ijobs_vectorizer.get_feature_names_out()
Ijobs_count_vect_df = pd.DataFrame(Ijobs_vector.todense(), columns=Ijobs_vector_features)
duration = time() - t0

In [21]:
jobs_rowSums = Ijobs_count_vect_df.copy()
for c in jobs_rowSums.columns:
  jobs_rowSums[c] = jobs_rowSums[c].apply(lambda x: min(1, x))

jobs_rowSums = jobs_rowSums.sum(axis=0)
jobs_rowSums.describe()

count     912.000000
mean       22.327851
std        87.331855
min         1.000000
25%         1.000000
50%         3.000000
75%        11.000000
max      1417.000000
dtype: float64

In [22]:
resume_rowSums = Iresume_count_vect_df.copy()
for c in resume_rowSums.columns:
  resume_rowSums[c] = resume_rowSums[c].apply(lambda x: min(1, x))

resume_rowSums = resume_rowSums.sum(axis=0)
resume_rowSums.describe()

count    912.000000
mean      36.532895
std       80.982259
min        1.000000
25%        3.000000
50%        9.000000
75%       31.250000
max      907.000000
dtype: float64